### Centroid creation
The goal of this notebook is to create 3 csv files (`c_10.csv`, `c_25.csv`, `c_25.csv`) which contains the centroids of our sampled states. In particular, these centroids will be calculated based on the following raw data:
- 10% (100K rows) of random sampled data.
- 90% (900K rows) of sampled data using baseline policy.

In [3]:
from os.path import join
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans


# Define our input folders and output files
DATADIR = join('..', 'data')

We read the files `ss_bsln_900k.csv` and `ss_random_100k.csv`. This is a database which contains 100K and 900K observations of the simulation of the agent playing with the opponent, using a random and baseline policy, respectively. 

In [4]:
ss_bsln = pd.read_csv(join(DATADIR, 'ss_bsln_900k.csv')).iloc[:, :8]
ss_random = pd.read_csv(join(DATADIR, 'ss_random_100k.csv')).iloc[:, :8]

We concat our dataframes which include both random and baseline samples and shuffle them. We also convert it to a numpy array.

In [5]:
ss = pd.concat([ss_bsln, ss_random], ignore_index=True).sample(frac=1).to_numpy()

We create 3 `MiniBatchKMeans` estimators with 5K, 10K and 25K clusters and a batch size of 2048 and fit it with our normalized sampled states dataframe. The purpose of this is to reduce the information of our sampled states in a number clusters, which will be the centroids of the RBF used in the next cells.

In [ ]:
mbkm_model_5 = MiniBatchKMeans(n_clusters=5_000, random_state=0, batch_size=2048, verbose=True)
mbkm_model_5.fit(ss)      # 2min

In [ ]:
mbkm_model_10 = MiniBatchKMeans(n_clusters=10_000, random_state=0, batch_size=2048, verbose=True)
mbkm_model_10.fit(ss)      # 4min

In [ ]:
mbkm_model_25 = MiniBatchKMeans(n_clusters=25_000, random_state=0, batch_size=2048, verbose=True)
mbkm_model_25.fit(ss)   # 21min

We save the information of the centroids from our files in a dataframe called `c_XX_mix`. The shape of this dataframe will be 5K, 10K or 25K (depending on the number of clusters selected in our algorithm) by 8 columns (number of dimensions of the states of our environment excluding the information of the opponent).

In [1]:
def save_centroids_file(model, name):
    head = 'x_agent,y_agent,xdot_agent,ydot_agent,' \
        'x_ball,y_ball,xdot_ball,ydot_ball'       
    np.savetxt(fname=join(DATADIR, name),
        X=model.cluster_centers_,
        fmt='%.5f',
        delimiter=',',
        header=head,
        comments='')

save_centroids_file(mbkm_model_5, "c_5_mix.csv")
save_centroids_file(mbkm_model_10, "c_10_mix.csv")
save_centroids_file(mbkm_model_25, "c_25_mix.csv")


NameError: name 'mbkm_model_5' is not defined